In [1]:
import numpy as np
import cvxpy as cp
from datetime import datetime
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

## Simulation Parameters

In [2]:
dt = 0.04  # timestep
t = 4  # duration
m = 1  # mass
q0 = np.array([0, 0])  # init pos
v0 = np.array([5, 0.5])  # init vel
force = np.array([0, 10])  # init force
g = 9.8  # gravitational acceleration
mu = 0.05  # coefficient of friction of the plane

## Simulation Backend

In [3]:
class SimSystem:
    def __init__(self, q0, v0):
        self.q = q0
        self.v = v0
    
    def step_q(self, update=True):
        """ Find next q according to system dynamics. Store if UPDATE. """
        raise NotImplementedError()
        
    def step_v(self, update=True):
        """ Find next v according to system dynamics. Store if UPDATE. """
        raise NotImplementedError()
    
    def plot(self, q=None):
        """
        Generate a plot for current config (or for Q if specified)
        """

In [4]:
class Block2D(SimSystem):
    def __init__(self, pos, vel, mass, force,
                 dt=0.1, mu=0.3, solver='ECOS'):
        super().__init__(pos, vel)
        self.m = mass
        self.f = force
        self.dt = dt
        self.mu = mu
        self.solver = solver
    
    def step_q(self, update=True):
        q_new = self.q + self.dt * self.v
        if update:
            self.q = q_new
        return q_new
    
    def step_v(self, update=True):
        b = self.get_b()
        v_new = self.v + (b - self.dt * self.f) / self.m
        if update:
            self.v = v_new
        return v_new
    
    def get_b(self):
        """
        b = argmin(v^\top v) s.t. b^\top b <= (\mu * m * g)
        """
        b = cp.Variable(self.v.shape[0])
        constraints = [cp.SOC(self.mu * self.m * g, b)]
        z = b - self.dt * self.f
        prob = cp.Problem(cp.Minimize(cp.norm(z, 2)**2 + 2 * self.m * (z @ self.v)),
                          constraints)
        prob.solve(solver=self.solver)
        self.b = b.value  # for debugging purposes
        return b.value
    
    def plot(self, q=None):
        if q is None:
            q = self.q
        fig = plt.figure()
        canvas = FigureCanvas(fig)
        plt.xlim(-8, 8)
        plt.ylim(-8, 8)
        plt.plot(q[0], q[1], 'ro')
        canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close()
        return img

## Run Simulation

In [5]:
def sim(system, timesteps=10, plot=False):
    """
    :param system: (SimSystem) initialized obj representing system,
                    including functions to update q and v
    :param timesteps: (int) number of timesteps to simulate for
    """
    qs, vs = [system.q.copy()], [system.v.copy()]
    if plot:
        plots = [system.plot()]
    t = 0
    while t < timesteps:
        qs.append(system.step_q().copy())
        vs.append(system.step_v().copy())
        if plot:
            plots.append(system.plot())
        t += system.dt
    return qs if not plot else (qs, plots)

In [6]:
system = Block2D(q0, v0, m, force, dt, mu)
qs, plots = sim(system, timesteps=t, plot=True)

## Plot

In [7]:
time = datetime.now().strftime('%m%d%y_%H%M%S')
imageio.mimwrite(f'sim2d_mass{m}.gif', plots, fps=int(1/dt));